# Install these librarires if you havent already

In [4]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# If you wanted to run streamlit main.py
## Run the below code block only

In [ ]:
!python -m streamlit run main.py 

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: No such option: --use-excel
